In [ ]:
from io import StringIO
import pandas as pd

# set the CSV raw data
sheet = """
EXPLAINER,date,spender,spend (x1k USD),spend_description,todate_spend (x1k USD),todate_spend_split (x1k USD),todate_ownership_percent_split
"This sheet tracks the expenses and informal ownership split for the 402 Prospect property. The principle followed is that ownership percentage equals the total spend percentage. There are 3 sections in this sheet: (1) the explainer section, (2) the expenses section, (3) the to-date section.

(1) The explainer section is this text and spans the column EXPLAINER.

(2) The expenses section is a list of expenses made by the owners towards the property. This part spans the columns [date, spender, spend, spend_description]. The expenses are sorted in descending time order (newest first).

(3) The to-date section expresses total to-date info about money spent and ownership-split. This section spans the columns [date, todate_spend, todate_spend_split, todate_ownership_percent_split]. Note that each row in this section summarizes all expenses made to-date i.e. each row expresses summary info covering the period from the oldest date in the sheet up to (and including) the date in that row.",2025-02-03,Harsh,4,mortgage 2025-02,,,
,2024-12-02,Aish,1.2,mortgage 2024-12 (gave Harsh Airbnb pay [Chloe $1115 12-02]),,,
"""

# read the CSV data into a DataFrame
sheet_df = pd.read_csv(StringIO(sheet))

print("\nsheet_df (original) =")
with pd.option_context("display.width", 1000, "display.max_colwidth", 30):
    print(sheet_df)


# fill the todate section of the sheet
def update_todate_cols_in_sheet(sheet_df):
    sheet_df = sheet_df.sort_values(by=["date"], ascending=True)
    sheet_df["spend (x1k USD)"] = sheet_df["spend (x1k USD)"].astype(float)
    cumulative_spend = 0
    cumulative_split = {}
    todate_spends = []
    todate_spend_splits = []
    todate_ownership_percent_splits = []
    for _, row in sheet_df.iterrows():
        spender = row["spender"]
        spend = row["spend (x1k USD)"]
        cumulative_spend += spend
        todate_spends.append(str(cumulative_spend))
        cumulative_split[spender] = cumulative_split.get(spender, 0) + spend
        todate_spend_splits.append(str(cumulative_split))
        todate_ownership_percent_splits.append(
            {
                spender: round(cumulative_split[spender] / cumulative_spend * 100, 1)
                for spender in cumulative_split
            }
        )
    sheet_df["todate_spend (x1k USD)"] = todate_spends
    sheet_df["todate_ownership_percent_split"] = todate_ownership_percent_splits
    sheet_df["todate_spend_split (x1k USD)"] = todate_spend_splits
    sheet_df = sheet_df.sort_values(by=["date"], ascending=False)
    return sheet_df


sheet_df = update_todate_cols_in_sheet(sheet_df)

print("\nsheet_df (todate section filled) =")
with pd.option_context("display.width", 1000, "display.max_colwidth", 30):
    print(sheet_df)


sheet_df (original) =
                       EXPLAINER        date spender  spend (x1k USD)              spend_description  todate_spend (x1k USD)  todate_spend_split (x1k USD)  todate_ownership_percent_split
0  This sheet tracks the expe...  2025-02-03   Harsh              4.0               mortgage 2025-02                     NaN                           NaN                            NaN 
1                            NaN  2024-12-02    Aish              1.2  mortgage 2024-12 (gave Har...                     NaN                           NaN                            NaN 

sheet_df (todate section filled) =
                       EXPLAINER        date spender  spend (x1k USD)              spend_description todate_spend (x1k USD) todate_spend_split (x1k USD) todate_ownership_percent_split
0  This sheet tracks the expe...  2025-02-03   Harsh              4.0               mortgage 2025-02                    5.2  {'Aish': 1.2, 'Harsh': 4.0}  {'Aish': 23.1, 'Harsh': 76.9}
1           